In [1]:
#!pip install fiona
import fiona
import pandas as pd
import shapely
import geopandas
from shapely.geometry import shape,mapping, Point, Polygon, MultiPolygon
import matplotlib.pyplot as plt

In [11]:
df = pd.read_csv('../data/yellow_tripdata_2021-01.csv', error_bad_lines=False)
dff = df.copy()

/var/folders/5z/fppys3dn27df6rctm0p78hq80000gn/T/ipykernel_53245/48116342.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('../data/yellow_tripdata_2021-01.csv', error_bad_lines=False)
/var/folders/5z/fppys3dn27df6rctm0p78hq80000gn/T/ipykernel_53245/48116342.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/yellow_tripdata_2021-01.csv', error_bad_lines=False)


In [7]:
# https://towardsdatascience.com/geospatial-operations-at-scale-with-dask-and-geopandas-4d92d00eb7e8
def assign_taxi_zones(df, lon_var, lat_var, locid_var):
    import numpy as np
    import geopandas
    from shapely.geometry import Point

    localdf = df[[lon_var, lat_var]].copy()
    
    localdf[lon_var] = localdf[lon_var].fillna(value=0.)
    localdf[lat_var] = localdf[lat_var].fillna(value=0.)
    
    shape_df = geopandas.read_file('../data/taxi_zones/taxi_zones.shp')
    shape_df = shape_df.to_crs({'init': 'epsg:4326'})
    try:
        local_gdf = geopandas.GeoDataFrame(
            localdf, crs={'init': 'epsg:4326'},
            geometry=[Point(xy) for xy in
                      zip(localdf[lon_var], localdf[lat_var])])

        local_gdf = geopandas.sjoin(
            local_gdf, shape_df, how='left', op='within')

        return local_gdf.OBJECTID.rename(locid_var)
    except ValueError as ve:
        print(ve)
        print(ve.stacktrace())
        series = localdf[lon_var]
        series = np.nan
        return series

In [ ]:
assign_taxi_zones(df,)

In [8]:
dff = dff.join(assign_taxi_zones(dff, " pickup_longitude", " pickup_latitude", "pickup_taxizone_id"))
dff = dff.join(assign_taxi_zones(dff, " dropoff_longitude", " dropoff_latitude", "dropoff_taxizone_id"))

KeyError: "None of [Index([' pickup_longitude', ' pickup_latitude'], dtype='object')] are in the [columns]"

In [9]:
dff.to_csv('result.csv',index=False)

In [6]:
dff.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2.0,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5
1,1.0,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2.0,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
2,1.0,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1.0,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0
3,1.0,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1.0,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0
4,2.0,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1.0,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5
